In [ ]:
import gym
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from copy import deepcopy
import collections
import tqdm

In [3]:
class ReplayBuffer():
    def __init__(self, maxlen):
        self.buffer = collections.deque(maxlen=maxlen)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        ind = np.random.choice(len(self.buffer), n, replace=False)
        s, a, r, d, ns = zip(*[self.buffer[i] for i in ind])
        return np.array(s), np.array(a), np.array(r), \
            np.array(d), np.array(ns)
    
    def size(self):
        return len(self.buffer)

In [4]:
class PrioritizedReplayBuffer():
    def __init__(self, maxlen, alpha, beta):
        self.buffer = collections.deque(maxlen=maxlen)
        self.alpha = alpha
        self.beta = beta
        self.weight = collections.deque(maxlen=maxlen)
    
    def put(self, transition):
        self.buffer.append(transition)
        if len(self.weight) > 0:
            self.weight.append(max(self.weight))
        else:
            self.weight.append(1.0)
    
    def sample(self, n):
        
        weight = np.array(self.weight) ** self.alpha
        prob = weight / weight.sum()
        ind = np.random.choice(len(self.buffer), n, p=prob, replace=True)
        s, a, r, d, ns = zip(*[self.buffer[i] for i in ind])
        imp_weight = (len(self.buffer) * prob[ind]) ** (-self.beta)
        imp_weight = imp_weight / imp_weight.max()

        return np.array(s), np.array(a), np.array(r), \
            np.array(d), np.array(ns), imp_weight, ind
    
    def update_priority(self, ind, td_err):
        update_weight = np.power(np.abs(td_err), self.alpha)

        for i in range(len(ind)):
            self.weight[ind[i]] = update_weight[i]
        
    def size(self):
        return len(self.buffer)

In [69]:
class PrioReplayBuffer:
    def __init__(self, maxlen, alpha, beta):
        self.maxlen = maxlen
        self.alpha = alpha
        self.beta = beta
        self.pos = 0
        self.buffer = []
        self.weight = np.zeros(maxlen)
    
    def put(self, transition):
        if len(self.buffer) < self.maxlen:
            self.buffer.append(transition)
        else:
            self.buffer[self.pos] = transition
            
        max_weight = max(self.weight.max(), 1e-6)
        self.weight[self.pos] = max_weight
        self.pos = (self.pos + 1) % self.maxlen
    
    def sample(self, n):
        if len(self.buffer) == self.maxlen:
            prios = self.weight
        else:
            prios = self.weight[:self.pos]
        
        probs = prios ** self.alpha
        probs /= probs.sum()
        indx = np.random.choice(len(self.buffer), n, p=probs)
        s, a, r, d, ns = zip(*[self.buffer[i] for i in indx])
        total = len(self.buffer)
        imp = (total * probs[indx]) ** (-self.beta)
        imp /= imp.max()
        return np.array(s), np.array(a), np.array(r), \
            np.array(d), np.array(ns), imp, indx
    
    def update_priority(self, indx, update_weight):
        for i in range(len(indx)):
            self.weight[indx[i]] = update_weight[i]
    
    def size(self):
        return len(self.buffer)

In [70]:
class Net(nn.Module):
    def __init__(self, input_dim=4, hid_dim=128, nlayers=2, out_dim=2):
        super(Net, self).__init__()
        self.model = []
        self.out_dim = out_dim
        inp = input_dim
        for i in range(nlayers):
            self.model.append(nn.Linear(inp, hid_dim))
            self.model.append(nn.ReLU())
            inp = hid_dim
        self.model.append(nn.Linear(inp, out_dim))
        self.model = nn.Sequential(*self.model)

    def forward(self, x):
        return self.model(x)

In [79]:
class DQN:
    def __init__(self, model, optim, n_step=2, gamma=0.9, update_freq=300):
        self.model = model
        self.optim = optim
        self.n_step = n_step
        self.gamma = gamma
        self.update_freq = update_freq
        self._cnt = 0
        self.model_target = deepcopy(self.model)
        self.model_target.eval()
    
    def learn(self, buffer, batch_size):
        self.model.train()
        
        s, a, r, d, ns, imp, ind = buffer.sample(batch_size)
        s = torch.from_numpy(s).float()
        a = torch.from_numpy(a).long()[:, None]
        ns = torch.from_numpy(ns).float()
        d = torch.from_numpy(d).float()[:, None]
        r = torch.from_numpy(r).float()[:, None]
        imp = torch.from_numpy(imp).float()[:, None]
        
        q_out = self.model(s)
        q_a = q_out.gather(1, a)
        
        # very important multiply d
        max_target_q = self.model_target(ns).max(dim=1)[0][:, None]
        target = r + self.gamma * max_target_q * d
        
        td_err = target.detach() - q_a
        loss = F.mse_loss(q_a, target.detach())
        buffer.update_priority(ind, (td_err**2).detach().squeeze().numpy())
        
        self.optim.zero_grad()
        loss.mean().backward()
        self.optim.step()
        self._cnt += 1
        
        if self._cnt % self.update_freq == 0:
            self.model_target.load_state_dict(self.model.state_dict())
    
    def sample_action(self, state, eps):
        self.model.eval()
        state = torch.from_numpy(state)
        with torch.no_grad():
            out = self.model(state)
            action = out.argmax().item()
        
        if np.random.rand() < eps:
            action = np.random.randint(len(out))
        
        return action

In [80]:
class Args:
    lr = 5e-4
    gamma = 0.98
    batch_size = 32
    max_episodes = 1000
    alpha = 0.6
    beta = 0.4

In [81]:
args = Args

In [ ]:
all_score_list = []
for i in range(10):
    env = gym.make('CartPole-v1')
    buffer = PrioReplayBuffer(5000, args.alpha, args.beta)
    lr = args.lr
    gamma = args.gamma
    batch_size = args.batch_size
    max_episodes = args.max_episodes

    n_obs = env.observation_space.shape[0]
    n_act = env.action_space.n
    model = Net(n_obs, hid_dim=128, nlayers=2, out_dim=n_act)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    policy = DQN(model, optimizer, gamma, update_freq=20)

    score_list = []
    for i in tqdm.tqdm(range(max_episodes)):
        score = 0
        length = 0
        s = env.reset()
        eps = max(0.01, 0.08 - 0.01*i/200)
        done = False
        while not done:
            a = policy.sample_action(s, eps)
            next_s, r, done, info = env.step(a)
            d = 0.0 if done else 1.0
            buffer.put((s, a, r/100, d, next_s))
            s = next_s

            score += r
            length += 1

        if buffer.size() > 2000:
            policy.learn(buffer, batch_size)

        score_list.append(score)
    all_score_list.append(score_list)